In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Tue Mar 29 20:16:13 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    33W / 250W |  15991MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!cat /etc/os-release

NAME="Ubuntu"
VERSION="18.04.5 LTS (Bionic Beaver)"
ID=ubuntu
ID_LIKE=debian
PRETTY_NAME="Ubuntu 18.04.5 LTS"
VERSION_ID="18.04"
HOME_URL="https://www.ubuntu.com/"
SUPPORT_URL="https://help.ubuntu.com/"
BUG_REPORT_URL="https://bugs.launchpad.net/ubuntu/"
PRIVACY_POLICY_URL="https://www.ubuntu.com/legal/terms-and-policies/privacy-policy"
VERSION_CODENAME=bionic
UBUNTU_CODENAME=bionic


In [ ]:
import tensorflow as tf
print(tf.version.VERSION)

2.8.0


In [ ]:
%pip install transformers

     |████████████████████████████████| 3.8 MB 8.1 MB/s 
     |████████████████████████████████| 6.5 MB 22.6 MB/s 
     |████████████████████████████████| 67 kB 2.4 MB/s 
     |████████████████████████████████| 895 kB 16.2 MB/s 
     |████████████████████████████████| 596 kB 58.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


**TensorFlow Benchmarks**

In [ ]:
import numpy as np
from PIL import Image as im
import time
from tqdm import tqdm
import argparse
import csv
import os
import tensorflow as tf
from tensorflow.keras import mixed_precision

try:
    from transformers import TFAutoModelForSequenceClassification
except Exception as e:
    print(e)


os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
tf.config.experimental.set_memory_growth(
    tf.config.list_physical_devices("GPU")[0], True
)

TIME_ID = time.strftime("%s")
DATA_DIR = f"./train/{TIME_ID}/"


def set_precision(dtype_policy):
    """Default to float32 for compute and variable dtypes. Optionally used float16 for compute, float32 for variable dtypes ("mixed_float16")"""
    if dtype_policy == "fp16":
        policy = mixed_precision.Policy("mixed_float16")
        mixed_precision.set_global_policy(policy)
    else:
        mixed_precision.set_global_policy("float32")


def set_xla(enable):
    """Use the accelerated linear algebra (xla)"""
    tf.config.optimizer.set_jit(enable)


def create_synthetic_data(data_dir, img_dim, img_per_label=128, num_labels=16):
    """Reading images in from the hard drive is more realistic than storing data entirely on GPU VRAM, or creating synthetic data at runtime"""
    os.makedirs(data_dir, exist_ok=True)
    for j in tqdm(range(num_labels), desc="Creating synthetic data", unit="folders"):
        os.makedirs(f"{data_dir}/{j}/")
        for i in range(img_per_label):
            _make_image(img_dim, f"{data_dir}/{j}/{j:03}_{i:06}_dummy.png")

    return data_dir


def _make_image(img_dim=224, filepath="./dummy.png"):
    """Helper function to actually create synthetic images for CNN training"""
    rng = np.random.default_rng()
    arr = rng.integers(low=0, high=256, size=(img_dim, img_dim, 3), dtype=np.uint8)
    img = im.fromarray(arr)
    img.save(filepath)


def cnn_create(model_name, img_shape, num_labels=16):
    if model_name == "resnet50":
        base_model = tf.keras.applications.resnet50.ResNet50(
            input_shape=img_shape, include_top=False, weights=None
        )
    elif model_name == "resnet50v2":
        base_model = tf.keras.applications.resnet_v2.ResNet50V2(
            input_shape=img_shape, include_top=False, weights=None
        )
    elif model_name == "resnet101v2":
        base_model = tf.keras.applications.resnet_v2.ResNet101V2(
            input_shape=img_shape, include_top=False, weights=None
        )
    elif model_name == "resnet152v2":
        base_model = tf.keras.applications.resnet_v2.ResNet152V2(
            input_shape=img_shape, include_top=False, weights=None
        )
    elif model_name == "mobilenetv2":
        base_model = tf.keras.applications.mobilenet_v2.MobileNetV2(
            input_shape=img_shape, include_top=False, weights=None
        )
    elif model_name == "inceptionv3":
        base_model = tf.keras.applications.inception_v3.InceptionV3(
            input_shape=img_shape, include_top=False, weights=None
        )
    elif model_name == "vgg16":
        base_model = tf.keras.applications.vgg16.VGG16(
            input_shape=img_shape, include_top=False, weights=None
        )
    elif model_name == "vgg19":
        base_model = tf.keras.applications.vgg19.VGG19(
            input_shape=img_shape, include_top=False, weights=None
        )
    else:
        exit()

    # Specify our own model top, to allow for arbitrary input shapes and num_labels
    inputs = tf.keras.Input(shape=img_shape)
    x = base_model(inputs, training=True)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    outputs = tf.keras.layers.Dense(num_labels, activation="softmax")(x)

    # Recompile model to train
    model = tf.keras.Model(inputs, outputs)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    )

    return model


def cnn_benchmark(train_ds, model, batch_size, epochs, warmup=True):
    # Warmup
    if warmup:
        model.fit(train_ds, epochs=2)

    # Benchmarking
    st = time.time()
    model.fit(train_ds, epochs=epochs)
    et = time.time()

    total_img = int(train_ds.__len__()) * batch_size * epochs
    throughput = round(total_img / (et - st), 1)
    print(f"{batch_size}: {throughput}")

    return throughput


def cnn_run(models, precision, xla):
    # runtime knobs
    set_precision(precision)
    set_xla(xla)

    # model knobs        
    SUPPORTED_MODELS = [
        "mobilenetv2",
        "resnet50",
        "resnet50v2",
        "resnet101v2",
        "resnet152v2",
        "vgg16",
        "vgg19",
    ]

    for model_name in models:
        assert model_name in SUPPORTED_MODELS

    BATCH_SIZES = [32, 64, 128, 256]
    img_dim = 224

    # benchmarking knobs
    epochs = 3
    warmup = True

    # Data logging file
    f = open(f"{TIME_ID}_results.csv", "w", buffering=1)
    fieldnames = [
        "model",
        "batch_size",
        "img_dims_xy",
        "precision",
        "xla",
        "throughput",
    ]
    writer = csv.DictWriter(f, fieldnames)
    writer.writeheader()

    # create synthetic data once
    create_synthetic_data(DATA_DIR, img_dim)

    # run benchmarks
    for model_name in models:
        model = cnn_create(model_name, (img_dim, img_dim, 3))
        for batch_size in BATCH_SIZES:
            print(f"TEST: {model_name} at b={batch_size}")
            train_ds = tf.keras.utils.image_dataset_from_directory(
                DATA_DIR, image_size=(img_dim, img_dim), batch_size=batch_size
            )
            try:
                throughput = cnn_benchmark(train_ds, model, batch_size, epochs, warmup)
            except tf.errors.ResourceExhaustedError as e:
                print(f"FAILED: {model_name} at b={batch_size} OOM")
                break

            row = [model_name, batch_size, img_dim, precision, xla, throughput]
            assert len(row) == len(fieldnames)
            writer.writerow({header: value for header, value in zip(fieldnames, row)})


def trans_create(model_name, seq_len=128, batch_size=32, steps=30, num_labels=8):
    # data
    dataset_size = batch_size * steps
    example_x = tf.random.uniform(
        (seq_len,),
        minval=0,
        maxval=1000,
        dtype=tf.dtypes.int64,
    )
    example_y = tf.random.uniform(shape=[], minval=0, maxval=num_labels, dtype=tf.int64)
    dataset_x = tf.stack([example_x] * dataset_size)
    dataset_y = tf.stack([example_y] * dataset_size)
    # model
    model = TFAutoModelForSequenceClassification.from_pretrained(
        model_name, num_labels=num_labels, cache_dir="./cache/"
    )
    model.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    )
    return {
        "sample_data": [dataset_x, dataset_y],
        "model": model,
    }

def trans_benchmark(model, dataset_x, dataset_y, batch_size, epochs, steps=30, warmup=True):
    # Warmup
    if warmup == True:
        _ = model.fit(x=dataset_x, y=dataset_y, batch_size=batch_size, epochs=2, verbose=2)

    # Benchmarking
    st = time.time()
    _ = model.fit(x=dataset_x, y=dataset_y, batch_size=batch_size, epochs=epochs, verbose=2)
    et = time.time()

    total_img = steps * batch_size * epochs
    throughput = round(total_img / (et - st), 1)
    print(f"{batch_size}: {throughput}")

    return throughput

def trans_run(models, precision, xla):
    # runtime knobs
    set_precision(precision)
    set_xla(xla)

    # model knobs
    SUPPORTED_MODELS = [
        "distilbert-base-uncased",
        "bert-large-uncased"
    ]

    for model_name in models:
        assert model_name in SUPPORTED_MODELS
    BATCH_SIZES = [16, 32, 64, 128, 256]

    # benchmarking knobs
    warmup = True
    steps = 2**6
    seq_len = 128
    epochs = 3

    # Data logging file
    f = open(f"{TIME_ID}_results.csv", "w", buffering=1)
    fieldnames = [
        "model",
        "batch_size",
        "seqlen",
        "precision",
        "xla",
        "throughput",
    ]
    writer = csv.DictWriter(f, fieldnames)
    writer.writeheader()

    # run benchmarks
    for model_name in models:
        for batch_size in BATCH_SIZES:
            print(f"TEST: {model_name} at b={batch_size}")
            try:
                train_items = trans_create(model_name, seq_len=seq_len, batch_size=batch_size, steps=steps) 
                model = train_items["model"]
                dataset_x, dataset_y = train_items["sample_data"]
                model.summary()
                throughput = trans_benchmark(model, dataset_x, dataset_y, batch_size, epochs=epochs, steps=steps, warmup=warmup)
                
                row = [model_name, batch_size, seq_len, precision, xla, throughput]
                assert len(row) == len(fieldnames)
                writer.writerow({header: value for header, value in zip(fieldnames, row)})
            except tf.errors.ResourceExhaustedError as e:
                print(f"FAILED: {model_name} at b={batch_size} OOM")
                break
    
if __name__ == "__main__":
    cnn_run(["resnet50v2"], "fp32", True)
    trans_run(["distilbert-base-uncased"], "fp32", True)


Creating synthetic data: 100%|██████████| 16/16 [00:18<00:00,  1.16s/folders]


TEST: resnet50v2 at b=32
Found 2048 files belonging to 16 classes.
Epoch 1/2
64/64 [==============================] - 43s 146ms/step - loss: 2.9290
Epoch 2/2
64/64 [==============================] - 10s 146ms/step - loss: 2.7545
Epoch 1/3
64/64 [==============================] - 10s 151ms/step - loss: 2.5997
Epoch 2/3
64/64 [==============================] - 10s 146ms/step - loss: 2.4517
Epoch 3/3
64/64 [==============================] - 10s 147ms/step - loss: 2.2174
32: 205.3
TEST: resnet50v2 at b=64
Found 2048 files belonging to 16 classes.
Epoch 1/2
32/32 [==============================] - 33s 974ms/step - loss: 1.2612
Epoch 2/2
32/32 [==============================] - 9s 263ms/step - loss: 0.1701
Epoch 1/3
32/32 [==============================] - 9s 262ms/step - loss: 0.0350
Epoch 2/3
32/32 [==============================] - 9s 263ms/step - loss: 0.0089
Epoch 3/3
32/32 [==============================] - 9s 263ms/step - loss: 0.0027
64: 217.0
TEST: resnet50v2 at b=128
Found 2048 fil

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/347M [00:00<?, ?B/s]

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_projector', 'vocab_transform', 'vocab_layer_norm', 'activation_13']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['dropout_19', 'pre_classifier', 'classifier']
You should probably TRAIN this model on a down-stream task to be able to use i

...

Model: "tf_distil_bert_for_sequence_classification_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 distilbert (TFDistilBertMai  multiple                 66362880  
 nLayer)                                                         
                                                                 
 pre_classifier (Dense)      multiple                  590592    
                                                                 
 classifier (Dense)          multiple                  6152      
                                                                 
 dropout_59 (Dropout)        multiple                  0         
                                                                 
Total params: 66,959,624
Trainable params: 66,959,624
Non-trainable params: 0
_________________________________________________________________
Epoch 1/2
64/64 - 34s - loss: 0.0234 - 34s/epoch - 533ms/step
Epoch 2/2
64/6

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_projector', 'vocab_transform', 'vocab_layer_norm', 'activation_13']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier', 'dropout_79', 'classifier']
You should probably TRAIN this model on a down-stream task to be able to use i

Model: "tf_distil_bert_for_sequence_classification_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 distilbert (TFDistilBertMai  multiple                 66362880  
 nLayer)                                                         
                                                                 
 pre_classifier (Dense)      multiple                  590592    
                                                                 
 classifier (Dense)          multiple                  6152      
                                                                 
 dropout_79 (Dropout)        multiple                  0         
                                                                 
Total params: 66,959,624
Trainable params: 66,959,624
Non-trainable params: 0
_________________________________________________________________
Epoch 1/2
64/64 - 52s - loss: 0.1730 - 52s/epoch - 806ms/step
Epoch 2/2
64/6

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_projector', 'vocab_transform', 'vocab_layer_norm', 'activation_13']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier', 'dropout_99', 'classifier']
You should probably TRAIN this model on a down-stream task to be able to use i

Model: "tf_distil_bert_for_sequence_classification_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 distilbert (TFDistilBertMai  multiple                 66362880  
 nLayer)                                                         
                                                                 
 pre_classifier (Dense)      multiple                  590592    
                                                                 
 classifier (Dense)          multiple                  6152      
                                                                 
 dropout_99 (Dropout)        multiple                  0         
                                                                 
Total params: 66,959,624
Trainable params: 66,959,624
Non-trainable params: 0
_________________________________________________________________
Epoch 1/2
FAILED: distilbert-base-uncased at b=256 OOM
